<a href="https://colab.research.google.com/github/spatank/CIS-700/blob/master/Homework%204/finetune_GPT2_LQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4 - Finetune GPT-2
This colab demonstrates how to fine-tune GPT-2 on a dataset of Lapham's Quarterly issues. We use the [Hugging Face Transformer](https://github.com/huggingface/transformers) library in order to do this.

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

## Setup

### Install HuggingFace Transfomers library.

In [1]:
!git clone https://github.com/huggingface/transformers

import os
os.chdir('/content/transformers')

!pip install .
!pip install -r ./examples/requirements.txt

os.chdir('/content/transformers/examples')

!pip install dict_to_obj

Cloning into 'transformers'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 20817 (delta 56), reused 82 (delta 47), pack-reused 20718
Receiving objects: 100% (20817/20817), 12.34 MiB | 28.45 MiB/s, done.
Resolving deltas: 100% (15040/15040), done.
Processing /content/transformers
     |████████████████████████████████| 3.7MB 4.7MB/s 
     |████████████████████████████████| 1.0MB 48.4MB/s 
     |████████████████████████████████| 870kB 51.9MB/s 
  Created wheel for transformers: filename=transformers-2.5.1-cp36-none-any.whl size=505540 sha256=9e924f2fe58efbbfb7c0147f7243dd60f09d2104f38a920067004d477da52c42
  Stored in directory: /tmp/pip-ephem-wheel-cache-pfow3ixe/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=49d147284f8bee6274787febe58b61f160584d50e3a212e31f4c94993a26c7f

In [2]:
import torch
import run_language_modeling
import run_generation
from dict_to_obj import DictToObj
import collections
import random
import numpy as np

### Mount your Google Drive
We will be saving trained checkpoints on your Google Drive so that they can be accessed even if the Colab session dies. Make sure to login with your UPenn credentials, as you will be saving several gigabytes of data, and Penn gives you unlimited Drive storage.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
!ls '/content/drive/My Drive/finetuned_models/laphams_quarterly/'

corpus_description.txt	    laphams_quarterly_train.txt  LQCorpus.txt
laphams_quarterly_test.txt  laphams_quarterly_valid.txt


### Lapham's Quarterly Data.
Split the data into a train, valid, and test set.

In [0]:
# from itertools import islice

# root_dir = '/content/drive/My Drive/finetuned_models/laphams_quarterly/'
# path_to_data = root_dir + 'LQCorpus.txt'
# file = open(path_to_data, encoding = 'UTF-8', errors = 'ignore').readlines()
# file_len = len(file)
# num_train_lines = round(0.90 * file_len)
# num_valid_lines = round(0.05 * file_len)
# num_test_lines = file_len - num_train_lines - num_valid_lines
# lines_in_sets = [num_train_lines, num_valid_lines, num_test_lines]
# temp = iter(file) 
# splits = [list(islice(temp, 0, ele)) for ele in lines_in_sets] 
# # split 0 is training data
# with open(root_dir + 'laphams_quarterly_train.txt', 'w') as f:
#   for line in splits[0]:
#     f.write('%s\n' % line)
# # split 1 is validation data
# with open(root_dir + 'laphams_quarterly_valid.txt', 'w') as f:
#   for line in splits[1]:
#     f.write('%s\n' % line)
# # split 2 is test data
# with open(root_dir + 'laphams_quarterly_test.txt', 'w') as f:
#   for line in splits[2]:
#     f.write('%s\n' % line)

In [6]:
!ls '/content/drive/My Drive/finetuned_models/laphams_quarterly/'

corpus_description.txt	    laphams_quarterly_train.txt  LQCorpus.txt
laphams_quarterly_test.txt  laphams_quarterly_valid.txt


## Finetune and Eval
The Hugging Face library provides a script [run_language_modeling.py](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) which contains all of the code for training and evaluating a language model.

We will be calling this script directly from the command line in order to launch training. We will also use functions from this script to conduct evaluation and generate samples at inference time.

### Launch fine-tuninng
We will be calling `run_language_modeling.py` from the command line to launch fine-tuning, **Running fine-tuning may take several hours.** Every `save_steps` steps, a checkpoint is saved to disk. The checkpoint contains all the learned weights for your model, and you can  always reload the model from a saved checkpoint, even if your Colab has crashed.

Below is an explanation of some of the arguments you might want to modify in the command below. 

* `--line_by_line`: Add `--line_by_line` if distinct lines of the text should be treated as distinct training examples. For example, if your dataset contains one story/tweet/article per line, this should be set.
* `--num_train_epochs`: The number of times to iterate over the train set. Increasing the number of epochs may result in better performance, but making this number too high will cause the model to overfit on the train set.
* `--block_size`: Your training text is truncated into blocks of this length. At test time, you will only want to generate sequences that are at most this length.
* `--gradient_accumulation_steps`: Update the model weights every this many steps. You shold set this to >1 when the batch size is very small to improve training stability.
* `--output_dir`: This is the where checkpoints will get saved. When you finetune on your own dataset, you should change this path. We recommend saving checkpoints to your Google Drive (`/content/drive/My Drive/`) so you can access them even if the Colab session dies.
* `--model_name_or_path` The path to the model weights to use when starting fine-tuning. You can set this to `gpt2-medium` to initialize with GPT-2's 355 million parameter model, or `gpt2` to initialize with their smaller 124 million parameter model. You can also set this to one of your own checkpoints to restart your training job if it crashes.

**I am getting out-of memory errors. What do I do?**

The number of trainable paramters in the model is a function of the `block_size` and the `batch_size`. If you are getting out-of-memory errors, then try drecreasing these value.

**Oh no! My computer went to sleep and the Colab disconnected.**

The train job might have still completed. Check the `output_dir` in your Google Drive to see if checkpoint files have been created there.

**Training is taking foreverrrrrr.**

Try decreasing `num_train_epochs` or changing `model_name_or_path` to `gpt2` instead of `gpt2-medium`.
If your evaluation set is very large, you might also want to remove the `evaluate_during_training` flag or increase `logging_steps`.

In [8]:
!python run_language_modeling.py \
    --output_dir='/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2-medium \
    --save_total_limit=5 \
    --num_train_epochs=1.0 \
    --do_train \
    --evaluate_during_training \
    --logging_steps=500 \
    --save_steps=500 \
    --train_data_file=/content/drive/My\ Drive/finetuned_models/laphams_quarterly/laphams_quarterly_train.txt \
    --do_eval \
    --eval_data_file=/content/drive/My\ Drive/finetuned_models/laphams_quarterly/laphams_quarterly_valid.txt \
    --per_gpu_train_batch_size=2 \
    --per_gpu_eval_batch_size=2 \
    --block_size=128 \
    --gradient_accumulation_steps=5

Streaming output truncated to the last 5000 lines.
Evaluating:  54% 210/392 [00:08<00:07, 24.81it/s]

Evaluating:  54% 213/392 [00:08<00:07, 24.73it/s]

Evaluating:  55% 216/392 [00:08<00:07, 24.71it/s]

Evaluating:  56% 219/392 [00:08<00:07, 24.66it/s]

Evaluating:  57% 222/392 [00:08<00:06, 24.66it/s]

Evaluating:  57% 225/392 [00:09<00:06, 24.70it/s]

Evaluating:  58% 228/392 [00:09<00:06, 24.74it/s]

Evaluating:  59% 231/392 [00:09<00:06, 24.79it/s]

Evaluating:  60% 234/392 [00:09<00:06, 24.80it/s]

Evaluating:  60% 237/392 [00:09<00:06, 24.79it/s]

Evaluating:  61% 240/392 [00:09<00:06, 24.77it/s]

Evaluating:  62% 243/392 [00:09<00:06, 24.78it/s]

Evaluating:  63% 246/392 [00:09<00:05, 24.76it/s]

Evaluating:  64% 249/392 [00:10<00:05, 24.77it/s]

Evaluating:  64% 252/392 [00:10<00:05, 24.55it/s]

Evaluating:  65% 255/392 [00:10<00:05, 24.62it/s]

Evaluating:  66% 258/392 [00:10<00:05, 24.70it/s]

Evaluating:  67% 261/392 [00:10<00:05, 24.76it/s]

Evaluating:  67% 264/392 [00:10

### Compute perplexity of a dataset.
This section shows how to compute perplexity of a dataset according to either the pre-trained or your fine-tuned language model. While this is possible to do by calling `run_language_modeling.py` on the command-line as above, we'll instead call the Python functions directly.

#### Look at what checkpoints are available
Run `ls` to look at what checkpoints saved been saved. You'll want to set `CHECKPOINT_PATH` below to one of these in order to evaluate the model weights saved in that checkpoint.

In [9]:
!ls '/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints'

checkpoint-1000  eval_results.txt   special_tokens_map.json  vocab.json
checkpoint-500	 merges.txt	    tokenizer_config.json
config.json	 pytorch_model.bin  training_args.bin


In [10]:
!ls '/content/transformers/'

CONTRIBUTING.md		     hubconf.py   notebooks  templates
deploy_multi_version_doc.sh  LICENSE	  README.md  tests
docker			     Makefile	  setup.cfg  transformers-cli
docs			     MANIFEST.in  setup.py   utils
examples		     model_cards  src	     valohai.yaml


#### Helper functions

In [0]:
def load_model(args):
  """Creates a model and loads in weights for it."""
  config_class, model_class, _ = run_language_modeling.MODEL_CLASSES[args.model_type]
  config = config_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  model = model_class.from_pretrained(
      args.model_name_or_path,
      from_tf=bool(".ckpt" in args.model_name_or_path),
      config=config,
      cache_dir=None,
  )
  model.to(args.device)
  return model

def set_seed(seed):
  """Set the random seed."""
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)

def do_perplexity_eval(args, model, data_file_path):
  """Computes the perplexity of the text in data_file_path according to the provided model."""
  set_seed(args.seed)

  args.eval_data_file=data_file_path

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  args.block_size = min(args.block_size, tokenizer.max_len)

  result = run_language_modeling.evaluate(args, model, tokenizer, prefix="")
  return result

#### Compute it.

In [13]:
# Set this to the checkpoint you want to evalute, or to "gpt2-medium" to
# evaluate the pre-trained model without finetuning.
# CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000'
CHECKPOINT_PATH = "gpt2-medium"

# Set this to the list of text files you want to evaluate the perplexity of.
DATA_PATHS = ["/content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_valid.txt",
              "/content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_test.txt"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  block_size = 128,
  local_rank=-1,
  eval_batch_size=2,
  per_gpu_eval_batch_size=2,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  line_by_line=False,
  overwrite_cache=None,
  model_type='gpt2',
  seed=42,
)
args = DictToObj(args)

model = load_model(args)

for data_path in DATA_PATHS:
  eval_results = do_perplexity_eval(args, model, data_path)
  perplexity = eval_results['perplexity']
  print('{} is the perplexity of {} according to {}'.format(
      perplexity, data_path, CHECKPOINT_PATH))

Running on device:  cuda


02/28/2020 08:37:30 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-config.json from cache at /root/.cache/torch/transformers/98aa65385e18b0efd17acd8bf64dcdf21406bb0c99c801c2d3c9f6bfd1f48f29.266bb9683aedfcb1f7006ad2e6894fce82b8dbbae8125f4fc8570b818005b83d
02/28/2020 08:37:30 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0

1258.7508544921875 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_valid.txt according to gpt2-medium


02/28/2020 08:38:38 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-vocab.json from cache at /root/.cache/torch/transformers/f20f05d3ae37c4e3cd56764d48e566ea5adeba153dcee6eb82a18822c9c731ec.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
02/28/2020 08:38:38 - INFO - transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-merges.txt from cache at /root/.cache/torch/transformers/6d882670c55563617571fe0c97df88626fb5033927b40fc18a8acf98dafd4946.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
02/28/2020 08:38:38 - INFO - run_language_modeling -   Loading features from cached file /content/drive/My Drive/finetuned_models/laphams_quarterly/gpt2_cached_lm_128_laphams_quarterly_test.txt
02/28/2020 08:38:38 - INFO - run_language_modeling -   ***** Running evaluation  *****
02/28/2020 08:38:38 - INFO - run_language_modeling -     Num 

1092.1812744140625 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_test.txt according to gpt2-medium


**Fine-tuned Model on LQ**

483.35565185546875 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_valid.txt according to /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000

428.5987243652344 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_test.txt according to /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000

**GPT-2 Without Fine-tuning**

1258.7508544921875 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_valid.txt according to gpt2-medium

1092.1812744140625 is the perplexity of /content/drive/My Drive/finetuned_models/laphams_quarterly/laphams_quarterly_test.txt according to gpt2-medium


### Generate samples
The following code generates text samples that are are continuations of a provided prompt.

In [0]:
def generate_samples(args, model, prompt_text):
  """Generating sampling for the provided prompt using the provided model."""
  set_seed(args.seed)

  _, _, tokenizer_class = run_language_modeling.MODEL_CLASSES[args.model_type]
  tokenizer = tokenizer_class.from_pretrained(args.model_name_or_path, cache_dir=None)

  requires_preprocessing = args.model_type in run_generation.PREPROCESSING_FUNCTIONS.keys()
  encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
  encoded_prompt = encoded_prompt.to(args.device)

  output_sequences = model.generate(
      input_ids=encoded_prompt,
      max_length=args.length + len(encoded_prompt[0]),
      temperature=args.temperature,
      top_k=args.k,
      top_p=args.p,
      repetition_penalty=args.repetition_penalty,
      do_sample=True,
      num_return_sequences=args.num_return_sequences,
  )

  # Remove the batch dimension when returning multiple sequences
  if len(output_sequences.shape) > 2:
    output_sequences.squeeze_()

  generated_sequences = []

  for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()

    # Decode text
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)

    # Remove all text after the stop token
    text = text[: text.find(args.stop_token) if args.stop_token else None]

    # Remove the excess text that was used for pre-processing
    text = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]

    # Add the prompt at the beginning of the sequence.
    total_sequence = prompt_text + text

    generated_sequences.append(total_sequence)

  return generated_sequences

In [16]:
# Set this to the checkpoint you want to use for generation, or to "gpt2-medium"
# to generate with the pre-trained model without finetuning.
CHECKPOINT_PATH = '/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000'
# CHECKPOINT_PATH = "gpt2-medium"

# You should try out other prompts as well as no prompt at all.
PROMPT = '<title=\"Quarterly\">'


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print("Running on device: ", device)

args = collections.defaultdict(
  model_name_or_path=CHECKPOINT_PATH,
  output_dir=CHECKPOINT_PATH,
  n_gpu=n_gpu,
  mlm=False,
  device=device,
  model_type='gpt2',
  seed=42,
  stop_token=None, # Set this if your dataset has a special word that indicates the end of a text.
  temperature=1.0,  # temperature sampling. Set this to temperature=1.0 to not use temperature.
  k=50,  # k for top-k sampling. Set this to k=0 to not use top-k.
  p=1.0,  # p for nucleus sampling. Set this to p=1.0 to not use nucleus sampling.
  repetition_penalty=None,
  length=100,  # Number of tokens to generate.
  num_return_sequences=100,  # Number of independently computed samples to generate.
)
args = DictToObj(args)

model = load_model(args)
sequences = generate_samples(args, model, PROMPT)
for idx, sequence in enumerate(sequences):
  print('\n====== GENERATION {} ======'.format(idx))
  print(sequence)

02/28/2020 08:42:49 - INFO - transformers.configuration_utils -   loading configuration file /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000/config.json
02/28/2020 08:42:49 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "embd_pdrop": 0.1,
  "eos_token_ids": null,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "num_beams": 1,
  "num_labels": 2,
  "num_return_sequences": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,

Running on device:  cuda


02/28/2020 08:43:39 - INFO - transformers.tokenization_utils -   Model name '/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming '/content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000' is a path, a model identifier, or url to a directory containing tokenizer files.
02/28/2020 08:43:39 - INFO - transformers.tokenization_utils -   Didn't find file /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000/added_tokens.json. We won't load it.
02/28/2020 08:43:39 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000/vocab.json
02/28/2020 08:43:39 - INFO - transformers.tokenization_utils -   loading file /content/drive/My Drive/finetuned_models/laphams_quarterly_checkpoints/checkpoint-1000/merges.txt
02/28/


====== GENERATION 0 ======
<title="Quarterly">quarterly print edition used permission usa verso copyright author used permission usa verso rights reserved ellington london david firth kent state university press copyright david firth used permission kent state university press used permission kent university press copyright david firth used permission kent university press kent university press copyright john adams robert goshaw london penguin books copyright robert goshaw houghton malcolm johnson houghton harcourt publishing group division pengu

====== GENERATION 1 ======
<title="Quarterly">breathe new breeze new music new style new times new style would be born baltimore new book born new book baltimore like every generation would become old one time another another new kind poetry would be born many new ways new things would also become old new times new sounds would come back new songs new style new things made old thing great make old great great new times new musical genre woul